<a href="https://colab.research.google.com/github/Karthik982018/Karthik982018/blob/main/cliff_walking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 

# COMP 532 – Machine Learning and Bio-Inspired Optimisation
# Assignment 1 Reinforcement Learning

# submitted by:
# Aby Tom - 201601516 - sgatom@liverpool.ac.uk
# Alwin Joseph Christopher - 201594340 - sgachri4@liverpool.ac.uk
# Jewel Sunny - 201603769 -	sgjsunny@liverpool.ac.uk
# Karthik Valiyaparambil Subramanian - 201586749 -  sgkvaliy@liverpool.ac.uk


# note:running the code in colab will fetch all the required results




# 



# importing openAI gym 
import gym
# importing the cliff walking environment
from gym.envs.toy_text.cliffwalking import CliffWalkingEnv
import random
import numpy as np
import matplotlib.pyplot as plt

# creating the environment
env=CliffWalkingEnv()

# defines the sarsa algorithm
# initAct is the initial action values for the states 
# env is the environment
# epsilon is the exploration coefficient
# discount is the discounting factor which is 1 since episodic
# episodes is the number of episodes
def sarsa(initAct,env,epsilon,step,discount,episodes):

  action_value=initAct
  rewards = np.empty(episodes)

  
  
  # iterating through the episode
  for i in range(episodes):
    
    # reset the state to init value
    env.reset()
    # current state
    state=env.s
    
    p=np.random.random()

    actionChoice=0
    # epsilon-greedy
    if p<epsilon:
      actionChoice =random.randint(0, 3)
      
    else:
      actionChoice=np.argmax(action_value[state])
    r=[]
    # defining the runs
    while True:
      # taking the next step
      next_state, reward, is_terminal, t_prob = env.step(actionChoice)
      p=np.random.random()
      a=0
      # epsilon greedy
      if p<epsilon:
        a=random.randint(0, 3)
      
      else:
        
        a=np.argmax(action_value[state])

      # updating the action value using sarsa
      action_value[state][actionChoice]+=step*(reward+(discount*action_value[next_state][a])-action_value[state][actionChoice])
      
      state=next_state
      actionChoice=a
      

      if is_terminal:
        break
        
      r.append(reward)

    sum=np.sum(r)     
    rewards[i]=sum
  # returning the reward and the final action value
  return rewards,action_value


# defines the q-learning algorithm
# initAct is the initial action values for the states 
# env is the environment
# epsilon is the exploration coefficient
# discount is the discounting factor which is 1 since episodic
# episodes is the number of episodes

def qLearning(initAct,env,epsilon,step,discount,episodes):

  action_value=initAct
  rewards = np.empty(episodes)

  
  
  # iterating through the episodes
  for i in range(episodes):
    
    # reset to the initial state
    env.reset()
    # current state
    state=env.s

    


    r=[]
    # defining the runs
    while True:
      
      
      p=np.random.random()
      a=0
      # epsilon greedy
      if p<epsilon:
        a=random.randint(0, 3)
      
      else:
        
        a=np.argmax(action_value[state])
      next_state, reward, is_terminal, t_prob = env.step(a)
      # updating the value using q-learning
      action_value[state][a]+=step*(reward+(discount*max(action_value[next_state]))-action_value[state][a])
      
      state=next_state
      
      

      if is_terminal:
        break


    sum=np.sum(r)     
    rewards[i]=sum
  # returning the rewards and the final action values
  return rewards,action_value
# smoothing the rewards
def smoothRewards(rewards):

  cum_rewards = []

  rewards_mean = np.array(rewards).mean()

  rewards_std = np.array(rewards).std()

  count = 0 # used to determine the batches

  cur_reward = 0 # accumulate reward for the batch

  for cache in rewards:

      count = count + 1

      cur_reward += cache

      if(count == 10):

          # normalize the sample

          normalized_reward = (cur_reward - rewards_mean)/rewards_std

          cum_rewards.append(normalized_reward)

          cur_reward = 0

          count = 0

  return cum_rewards

# initialise the action values and and runs sarsa and q-learning and plots the output
def run(env,epsilon,step,discount,episodes):
  # initialising the action values
  action_value_sarsa=[]
  np.random.seed(13)
  for i in range(0,env.nS):
    qa=[]
    for j in range(env.nA):
      r=np.random.random()
      qa.append(r)
    action_value_sarsa.append(qa)
  term_action=[]
  for i in range(env.nA):
    term_action.append(0)
  
  

  

  action_value_sarsa.append(term_action)
  action_value_qlearning=np.copy(action_value_sarsa)
  # run sarsa
  sarsaRewards,average1=sarsa(action_value_sarsa,env,epsilon,step,discount,episodes)
  # run q-learning
  qRewards,average2=sarsa(action_value_qlearning,env,epsilon,step,discount,episodes)
  
  # applying smoothening
  cumSarsa=smoothRewards(sarsaRewards)
  cumQLearn=smoothRewards(qRewards)

  # plotting
  plt.figure(figsize = (12, 8))
  plt.title("epsilon value:{} ".format(epsilon))

  plt.xlabel("batches of episodes(sample size:10)")
  plt.ylabel("cumulative rewards")

  plt.plot(cumSarsa, label ='sarsa')
  plt.plot(cumQLearn, label ='Q-learning')

  plt.legend()
  plt.show()
     

run(env,.01,.1,1,500)
run(env,.1,.1,1,500)
run(env,.2,.1,1,500)

      
